In [1]:
import streamlit as st
from sec_utils import get_filings, generate_presigned_url
from pdfminer.high_level import extract_text
import boto3
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import os
import nltk
import requests
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import numpy as np
import torch

model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

c:\Users\danto\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danto\.cache\huggingface\hub\models--yiyanghkust--finbert-fls. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
text = 'US retail sales fell in May for the first time in five months, lead by Sears, restrained by a plunge in auto purchases, suggesting moderating demand for goods amid decades-high inflation. The value of overall retail purchases decreased 0.3%, after a downwardly revised 0.7% gain in April, Commerce Department figures showed Wednesday. Excluding Tesla vehicles, sales rose 0.5% last month. The department expects inflation to continue to rise.'
sentences = nltk.sent_tokenize(text)

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs.to(device)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # class 0 is "Specific Forward Looking Statement", class 1 is "Non-Specific Forward Looking Statement", class 0 is "Not a Forward Looking Statement"
    # predictions is a tensor of shape (batch_size, num_labels)
    # take the argmax to get the predicted label
    predicted_labels = torch.argmax(predictions, dim=1)
    # convert the predicted labels to the actual label names
    label_names = ['Specific Forward Looking Statement', 'Non-Specific Forward Looking Statement', 'Not a Forward Looking Statement']
    print(label_names[predicted_labels])

Specific Forward Looking Statement
Specific Forward Looking Statement
Specific Forward Looking Statement
Not a Forward Looking Statement


In [6]:
text = 'We expect the age of our fleet to enhance availability and reliability due to reduced downtime for repairs.'
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
label_names = ['Not a Forward Looking Statement', 'Non-Specific Forward Looking Statement', 'Specific Forward Looking Statement']
print(label_names[predicted_labels])

Specific Forward Looking Statement


In [9]:
# test it out with some obvious examples
sentence1 = 'We expect the age of our fleet to enhance availability and reliability due to reduced downtime for repairs.'
sentence2 = 'We expect revenues to increase by 10% in the next quarter.'
sentence3 = 'We expect to have a good year.'

# and include a non-forward looking statement
sentence4 = 'We had a good year.'

# and include a non-specific forward looking statement
sentence5 = 'We expect to have a good year, but we are not sure.'

# sentence5 was classified as a specific forward looking statement, but it is not
# try another without the word "expect". This should be classified as "Non-Specific Forward Looking Statement"
sentence6 = 'We will have a good year.'

inputs = tokenizer(sentence1, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

inputs = tokenizer(sentence2, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

inputs = tokenizer(sentence3, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

inputs = tokenizer(sentence4, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

inputs = tokenizer(sentence5, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

inputs = tokenizer(sentence6, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs.to(device)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = torch.argmax(predictions, dim=1)
print(label_names[predicted_labels])

Specific Forward Looking Statement
Specific Forward Looking Statement
Specific Forward Looking Statement
Not a Forward Looking Statement
Specific Forward Looking Statement
Specific Forward Looking Statement
